<a href="https://colab.research.google.com/github/tonytani37/stats/blob/main/%E3%82%B9%E3%82%BF%E3%83%83%E3%83%84%E3%83%87%E3%83%BC%E3%82%BF%E4%BD%9C%E6%88%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# まずは前提条件
Googleアカウントが必須です。

ブラウザは最新のChromeの利用を強く推奨します。(Microsoft Edgeの最新版は大丈夫ですが、古いEdgeやIEは動かないかもしれません。FirefoxやSafariでは動くと思います）

## 注意事項
### 処理自体は膨大な能力を持つGoogleクラウド上で実行されるので、目の前のパソコンが多少古くてもあまり心配することはありません。

１．説明を読んで、上から順番に番号順に処理をしてください。

２．処理はそれぞれの機能ごとに分けてありますので、今後修正したい場合にはその部分だけ差し替えればオーケーです。

３．処理によっては実行するとその下によくわからない文字がずらずらと表示されるものとされないものがあります。どのタイミングで処理が終わったかわかりづらいかもしれないので、各処理が完了すると「x おわり」と表示するようにしていますで、そこまで我慢してください。（表示されるメッセージのnは処理の番号です）




# さあ、はじめましょう
ここから処理が始まります。

### 最初にデータ化したいPDFファイルを入手します
1-1 は自分のPCにあるPDFファイルを変換したい場合に使います。

1-2 はよくわからないけどとりあえず練習用に試してみたい人向けに、Xリーグのホームページにある2019/8/24の富士通フロンティアーズ対IBM BigBlueの対戦のものがダウンロードされます。

どちらか一つ、好きな方を実行してください。（1-1の場合は、以降使われるPDFファイル名や画像データ名が以下にあるものと異なるので、適宜変更してくださいね）

# 初回の実行時に「これはGoogleが提供してないけど実行する？」的なメッセージが出るかも知れませんが、気にせず実行して下さい。

その１）自分のPCにあるPDFファイルを変換したい場合

In [ ]:
# 1-1
#OCRであつかうPDFファイルを読み込む（変換したいPC上のファイルを指定してくだされ）
from google.colab import files
f = files.upload()
#読み込んだ画像のファイル名
filename = list(f)[0]

print('1-1 おわり')

その２）とりあえず試してみたい場合　

In [3]:
# 1-2
#OCRであつかうPDFファイルを読み込む（練習用に特定のPDFファイルをダウンロードしてくる）
!rm -r /content/pdf/
!mkdir /content/pdf/

! wget -c -P '/content/pdf/' 'https://xleague.s3.ap-northeast-1.amazonaws.com/wp-content/uploads/2019/07/20190824_X1_FF_BB_921.pdf'
# filename = "20190824_X1_FF_BB_921.pdf"

print('1-2 おわり')

rm: cannot remove '/content/pdf/': No such file or directory
--2021-05-22 07:20:44--  https://xleague.s3.ap-northeast-1.amazonaws.com/wp-content/uploads/2019/07/20190824_X1_FF_BB_921.pdf
Resolving xleague.s3.ap-northeast-1.amazonaws.com (xleague.s3.ap-northeast-1.amazonaws.com)... 52.219.0.255
Connecting to xleague.s3.ap-northeast-1.amazonaws.com (xleague.s3.ap-northeast-1.amazonaws.com)|52.219.0.255|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6365669 (6.1M) [application/pdf]
Saving to: ‘/content/pdf/20190824_X1_FF_BB_921.pdf’

20190824_X1_FF_BB_9 100%[===================>]   6.07M  16.9MB/s    in 0.4s    

2021-05-22 07:20:44 (16.9 MB/s) - ‘/content/pdf/20190824_X1_FF_BB_921.pdf’ saved [6365669/6365669]

1-2 おわり


### PDFを画像へ変換します

In [4]:
# 2
# PDFから画像へ変換するためのライブラリ、ツールを準備する
!apt-get install poppler-utils 
!pip install pdf2image

print('2 おわり')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 1s (120 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
2 おわり


In [5]:
# 3
# 画像データに変換する処理
from pathlib import Path
from pdf2image import convert_from_path
import glob

# Google Colabのデフォルト環境内にimageというフォルダを作る
!rm -r /content/image/
!mkdir /content/image/

def pdf2img(filename):
    pdf_path = Path(filename)
    #outputのファイルパス（この場合は上でつくったimageフォルダを指定）
    img_path=Path("/content/image/")

    #変換されたjpegファイルがimg_pathで指定されたフォルダに保存されます
    convert_from_path(pdf_path, output_folder=img_path,fmt='jpeg',output_file=pdf_path.stem)

#フォルダにあるPDFファイルの一覧を取得する
pdf_list = glob.glob('/content/pdf/*.pdf')

for row in pdf_list:
    pdf2img(row)

print('3 おわり')

rm: cannot remove '/content/image/': No such file or directory
3 おわり


### 以上で画像変換完了。PDFに複数ページがある場合、ページごとに画像データ（ファイル名末尾が連番）が作成されるので、一つのPDFファイルから複数の画像ファイルが作成されたとしても、驚かないこと。

### ここからOCR読み込み処理です。まずは準備しましょう
最初にOCR読み込み用のライブラリとツールをインストールします

In [9]:
# 4
# Tesseract OCRの関連ライブラリを準備する
# OCR読み込みに必要なツールをインストールします
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pyocr

#日本語トレーニングデータのダウンロードとして環境設定します
!curl -L https://github.com/tesseract-ocr/tessdata/raw/master/jpn.traineddata > jpn.traineddata
!cp jpn.traineddata /usr/share/tesseract-ocr/4.00/tessdata

# PythonでOCR読み込み変換に必要なライブラリを準備します
from PIL import Image
import pyocr
import pyocr.builders

print('4 おわり')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   145  100   145    0     0    432      0 --:--:-- --:--:-- --:--:--   434
100 34.0M  100 34.0M    0   

In [ ]:
 # 5
 #日本語トレーニングデータのダウンロードとして環境設定します
!curl -L https://github.com/tesseract-ocr/tessdata/raw/master/jpn.traineddata > jpn.traineddata
!cp jpn.traineddata /usr/share/tesseract-ocr/4.00/tessdata
print('5 おわり')

In [ ]:
# 6
# PythonでOCR読み込み変換に必要なライブラリを準備します
from PIL import Image
import pyocr
import pyocr.builders

print('6 おわり')

### OCR読み込みの前に、必要なツールがインストールされているか確認しませう。
うまくいかない場合は環境構築が間違っているので、最初からもう一度がんばろう。


OCR tool is 'Tesseract (sh)' と表示されればオーケー

In [10]:
# 7
#OCRが使用可能かの設定状態チェックしてみる
tools = pyocr.get_available_tools()
if len(tools) == 0:
    print("OCR tool is not found")
    sys.exit(1)

#OCRツール名を表示
tool = tools[0]
print("OCR tool is '%s'" % (tool.get_name()))

print('7 おわり')

OCR tool is 'Tesseract (sh)'
7 おわり


Available languages:　の次に　jpn　が入ってればオーケー（その他 osd,engなどもあるはず。もしかするとないものもあるかもしれないけど、とりあえず jpn が入っていればオーケー）

In [11]:
# 8
#OCR対応言語を表示して'jpn'が入っているか確認してみる
langs = tool.get_available_languages()
print("Available languages: %s" % ", ".join(langs))
lang = langs[0]
print("Will use lang '%s'" % (lang))

print('8 おわり')

Available languages: eng, jpn, osd
Will use lang 'eng'
8 おわり


### ここから実際のOCR読み込み処理です。
ここでは、fnameという変数にファイルのパスをセットしています。Google Colaboでは画面左側に作成されたファイルを扱うので、デフォルトの'/content/以降の部分に読み込みたい画像ファイル名(pathも正しく指定してね)を指定してください。ファイル名とパスの指定は、読み込みたいファイルを右クリックすると「パスをコピー」という項目があるので、そいつをクリックしてコピーし、fname = 以降に”または’のどちらかで囲って、その間にペーストすればオーケーです。

(本当は特定のフィルだにある画像ファイルを一括で処理できればいいんだろうけれど、そういうやつは次に考えます）

In [43]:
# 9
#読み込んだ画像をOCRでテキスト抽出。

#うまく変換できなかった文字を強制的に置き換える
conv_text = {"①":"1","②":"2","③":"3","④":"4","⑤":"5","⑥":"6","⑦":"7","⑧":"8","⑨":"9",
             "⑩":"10","⑪":"11","⑫":"12","⑬":"13","⑭":"14","⑮":"15",
             "⑯":"16","⑰":"17","⑱":"18","⑲":"19","⑳":"20",',':'',"&":"","-":"",
             "PASS":"PASS","RUN":"RUN","FG":"FG","PUNT":"PUNT","し":"L","、":" "}

!rm -r /content/ocr/
!mkdir /content/ocr/

def ocr2txt(fname):
    txt = tool.image_to_string(
        Image.open(fname),
        lang="jpn",
        builder=pyocr.builders.TextBuilder(tesseract_layout=6)
        )
    # プレーに関係ありそうな文字として　"&",'Penalty','Kick-off','Extra Point','TIMEOUT','Quarter'が含まれる行だけ抽出する
    # そのほかの特定文字も抽出対象としたい場合は ifの次に or ('' & in line) を追加挿入し、''の間に含めたい文字列を指定するとできますよ
    return  [line for line in txt.splitlines() if ('&' in line) or ('Penalty' in line) or ('Kick-off' in line) or ('Extra Point' in line) or ('TIMEOUT' in line) or (':' in line) or ('Quarter' in line)]
    
def num_conv(line):
    for ct in conv_text:
        line = line.replace(ct,conv_text[ct])
    return line

#フォルダに出力された画像ファイルの一覧を取得
jpg_list = glob.glob('/content/image/*.jpg')

#画像ファイルをからOCRでデータ化したものを一つのファイルにまとめる
output_file = []
for row in sorted(jpg_list):
    output_file.append([row1.replace(' ','') for row1 in [num_conv(out_txt) for out_txt in ocr2txt(row)]])

output_txt = ''
for x in output_file:
    for z in x:
        output_txt += z+'\n'


#最後にテキストファイルを出力する
output_file_txt = '/content/ocr/'+jpg_list[0][15:23]
with open(output_file_txt,mode='w') as f:
    f.write(output_txt)

print('9 おわり')

9 おわり


In [109]:
import re
import glob
import pandas as pd 

filename = glob.glob('/content/ocr/*')[0]

with open(filename,mode='r',encoding='UTF-8') as f:
    file_in = f.readlines()

in_list = [row for row in file_in]

def find(s,q):
    # fp = re.search(r'\w\w\d+[LMR]', s)
    # if fp is not None:
    qt = q
    fp_all = fp.group()
    fieldpos = fp_all[:2]
    y = re.search(r'\d+',fp_all[2:])
    yardline = y.group()
    hash = fp_all[-1]
    down = s[:1] 
    tg = re.search(r'\d\d',s[2:])
    if tg is not None:
        togo = tg.group()
    else:
        togo = 0
    yd = re.search(r'-*\d+y',s)
    if yd is not None:
        if 'ロス' in s:
            yards = 0 - int(yd.group()[:-1])
        else:
            yards = yd.group()[:-1]
    else:
        yards = 0
    if 'RUN' in s:
        play = 'RUN'
    elif 'PASS' in s:
        play = 'PASS'
    elif 'PUNT' in s:
        play = 'PUNT'
    else:
        play = 'None'
    other = s.replace(',','')
    other = other.replace('\n','')
    return qt,int(down),int(togo),fieldpos,int(yardline),hash,play,int(yards),other


def set_q(s):
    # quarter = re.search(r'Quarter', s)
    # if quarter:
    fq,sq,tq,yd = '','','',''
    fq = re.search(r'First', s)
    sq = re.search(r'Second', s)        
    tq = re.search(r'Third', s)        
    yq = re.search(r'Fourth', s)
    if fq:
        q = 1
    elif sq:
        q = 2
    elif tq:
        q = 3
    elif yq:
        q = 4
    return q

lines_strip_kekka = []
for s in in_list:
    quarter = re.search(r'Quarter', s)
    if quarter:
        q = set_q(s)
    fp = re.search(r'\w\w\d+[LMR]', s)
    if fp:
        f = find(s,q)
    else:
        f = s[:-1],0,0,0,0,0,0,0
    if f is not None:
        lines_strip_kekka.append(f)

kekka = pd.DataFrame(lines_strip_kekka,columns=['quarter','down','togo','fieldposition','yardline','hash','play','yards','other'])
kekka.to_csv('test',index=False)

In [59]:
df = pd.read_csv('/content/test')
# df = df[(df['offence']=='FF') | (df['offence']=='BB')]
df

,quarter,down,togo,fieldposition,yardline,hash,play,yards,other
0,2019/08/24(土)会場:富士通スタジアム川崎,0,0,0,0,0,0,0,NaN
1,PlaybyPlayFirstQuarter,0,0,0,0,0,0,0,NaN
2,FF35Kickoff#25大塚健一キック…BB13#81栗原嵩35yリターン(#23奥田凌...,0,0,0,0,0,0,0,NaN
3,IBMBigBlue12:00,0,0,0,0,0,0,0,NaN
4,4,1,48,BB,48,L,PASS,0,110BB48LPASS#2政本悠紀パス失敗
...,...,...,...,...,...,...,...,...,...
215,4,4,18,FF,18,M,PUNT,0,44FF18MPUNT_#39吉田元紀パントポールデッド
216,IBMBigBlue0:31,0,0,0,0,0,0,0,NaN
217,4,1,36,0F,36,R,RUN,9,110F36RRUN約7山中大輔9yラン(初3井本健一朗)
218,富士通フロンティアーズ556:14430717211,0,0,0,0,0,0,0,NaN


### おまけのおまけ
この処理を上から番号順に実行させればGoogle Colabo環境(中身はJupyter notebook)でPDFデータから文字を抜き出す処理まではできるようになっています。

で、データ分析のためのデータ整備の本番はこれからになりますが、この時点ですでに印刷された文字をデータ化できているので、ファイルをダウンロードした後に、自分で項目を分割してやればエクセルなどパソコン上でデータとして扱えるようになります。

### 注意）ダウンロードしたファイルはまだ整備されていないので、このままで素直にエクセル等で開くことは出来ないかも知れません。まずはメモ帳などで修正してからエクセル等で開いて下さい。

処理を自動化することは作業の効率化という点で非常に大切ですが、意味が分からずやるよりも最初はデータ構造を（なんとなくでも）理解しながら作業を行った方が、後々データの見直しや分析のやり方を見直したいときのアイディアがひらめく可能性が高くなると思います。

今回ダウンロードできるファイル(stats.txt)はパソコンのメモ帳で開けば、普通の文字列として読んだり修正したりすることができます。分割したい項目をカンマ(,)で区切ってあげて保存した後にファイル名の拡張子をCSVにすれば、それはもう立派なCSVデータです。（CSVは　Comma-Separated Valuesの略で、文字通りカンマで項目が区切られているデータというだけの意味です。拡張子自体はパソコン自体が処理を楽にするためにつけられている文字列というだけなので、自分が処理させたいプログラムに紐づけられたものに勝手に変えてもオーケーです）

## とりあえず今回の最後に
現在、機械学習などが活用されその精度は上がってきてはいるものの、OCR処理で読み取り間違い発生をゼロにすることはなかなか難しい状況です。特に今回のような数字と記号、漢字とアルファベットが混ざったものをプログラム処理だけですべてきれいなデータが作成できないと考えた方が良いと思います。（元データ自体、半角と全角の数字が混ざっているかもしれないし）

そのため、結局最後は人間が目で確認しながら手で微調整をする必要があります。今回はそれほど大きいデータ量ではないので、心の中では初めから手入力でCSVファイルを作成したほうが良いかもともしれないなと思います。しかし、一からデータ登録をすることを考えるとやる気がなくなってしまうので、こういった下準備が気軽にできる環境が生み出されてきていることには大きな意義があることは確かです。

だだそれ以前に、データ提供されている側にはPDFデータを作るためにエクセルなどで準備しているんじゃないかと思われ、ならばそのデータを（PDFの参考データとしてでも）公開してくれれば、少なくともOCR処理での読み取り間違いなどの発生はなくなり、よりデータを活用しやすくなります。

しかしながら、現在のデータではPASSやPUNTなどの投げたりけったりした選手と、それを受けたりリターンしたりした選手の区別などデータの整理整頓（標準化）ができておらず、突っ込んだ分析や機械学習にこのデータを使おうとした場合にはまだまだ問題は山積状態です。

そんな五里霧中な状態ではあるものの、我々にはNFLという偉大な先達がおられます。彼らがこれまで積み上げてきたノウハウの端っこでも参考にしながら、もっとデータ活用できる環境が構築されることの切望します。

※もしトップレベルのチームではそういったデータ取得、活用ノウハウが共通化されているならば、それについて教えてください。

一度に理想まで近づくことはできませんが、まずは手始めに野球のスコアブックのようなデータ採取の標準フォーマットを決めて、それが各チームで広く使われるようにすることが必須でしょう。そうすれば、それを基にしたツール開発や、データ自体の流通や活用が円滑に行わえるようになると考えます。

これからもっと「データドリブン（データをもとに物事を進めてゆく）」な世の中になると思います。身近なことからコツコツと、できることからやってみましょう。